In [75]:
import os
import pandas as pd
from constants import *
import hashlib

In [76]:
# Define o caminho para a pasta "dados"
data_folder = "dados"

# lista para armazenar os dataframes de cada arquivo CSV
dfs = []

# Percorre todos os arquivos .txt na pasta
for filename in os.listdir(data_folder):
    if filename.endswith('.txt'):
        # lê o arquivo como CSV com o pandas
        df = pd.read_csv(os.path.join(data_folder, filename), delimiter=';')

        # Verificar se o arquivo possui a coluna 'competênciaexc'
        if 'competênciaexc' not in df.columns:
            # Criar a coluna 'competênciaexc' com valor nulo
            df['competênciaexc'] = None

        # Verificar se o arquivo possui a coluna 'indicadordeexclusão'
        if 'indicadordeexclusão' not in df.columns:
            # Criar a coluna 'indicadordeexclusão' com valor 0
            df['indicadordeexclusão'] = 0
            
        # Criar a coluna data_mov
        df['competênciamov'] = df['competênciamov'].astype(str) + '01'
        df['data_mov'] = pd.to_datetime(df['competênciamov'], format='%Y%m%d')

        # criar colunas nu_ano e nu_mes
        df['nu_ano'] = df['data_mov'].dt.year
        df['nu_mes'] = df['data_mov'].dt.month

        # Alterar tipo de dados
        df['idade'] = df['idade'].fillna(0).astype(int)
        df['salário'] = df['salário'].str.replace(',', '.').fillna(0).astype(float)
        df['horascontratuais'] = df['horascontratuais'].str.replace(',', '.').fillna(0).astype(float)

        # Realizar de-para
        df['graudeinstrução'] = df['graudeinstrução'].astype(str).apply(lambda x: map_grau_instrucao.get(x, "Não Identificado"))
        df['sexo'] = df['sexo'].astype(str).apply(lambda x: map_genero.get(x, "Não Identificado"))
        df['raçacor'] = df['raçacor'].astype(str).apply(lambda x: map_etnia.get(x, "Não Identificado"))
        df['tipodedeficiência'] = df['tipodedeficiência'].astype(str).apply(lambda x: map_tp_deficiencia.get(x, "Não Identificado"))
        df['tipoempregador'] = df['tipoempregador'].astype(str).apply(lambda x: map_tp_empregador.get(x, "Não Identificado"))
        df['tipoestabelecimento'] = df['tipoestabelecimento'].astype(str).apply(lambda x: map_tp_estabelecimento[x])
        df['seção'] = df['seção'].astype(str).apply(lambda x: map_secao_economica.get(x, "Não Identificado"))
        df['subclasse'] = df['subclasse'].astype(str).apply(lambda x: map_subclasse_econ.get(x, "Não Identificado"))
        df['tamestabjan'] = df['tamestabjan'].astype(str).apply(lambda x: map_faixa_tam_estab.get(x, "Não Informado"))
        df['tipomovimentação'] = df['tipomovimentação'].astype(str).apply(lambda x: map_tp_movimentacao.get(x, "Não Identificado"))
        df['categoria'] = df['categoria'].astype(str).apply(lambda x: map_categoria.get(x, "Não Identificado"))
        df['cbo2002ocupação'] = df['cbo2002ocupação'].astype(str).apply(lambda x: map_cbo_ocupacao.get(x, "Não Identificado"))
        df['indtrabintermitente'] = df['indtrabintermitente'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indtrabparcial'] = df['indtrabparcial'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indicadoraprendiz'] = df['indicadoraprendiz'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indicadordeexclusão'] = df['indicadordeexclusão'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indicadordeforadoprazo'] = df['indicadordeforadoprazo'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))

        # Concatenar as colunas do DataFrame
        concatenacao_trabalhador = df['graudeinstrução'] + df['sexo'] + df['idade'].astype(str) + df['raçacor'] + df['tipodedeficiência']
        # Calcular o hash para a concatenação das colunas
        hashes_trabalhador = concatenacao_trabalhador.apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        # Adicionar a coluna 'hash' ao DataFrame
        df['sk_trabalhador'] = hashes_trabalhador

        # Concatenar as colunas do DataFrame
        concatenacao_empregador = df['tipoempregador'] + df['tipoestabelecimento'] + df['seção'] + df['subclasse'] + df['tamestabjan']
        # Calcular o hash para a concatenação das colunas
        hashes_empregador = concatenacao_empregador.apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        # Adicionar a coluna 'hash' ao DataFrame
        df['sk_empregador'] = hashes_empregador

        # Concatenar as colunas do DataFrame
        concatenacao_movimentacao = df['tipomovimentação'] + df['categoria'] + df['cbo2002ocupação'] + df['indtrabintermitente'] + df['indtrabparcial'] + df['indicadoraprendiz'] + df['indicadordeexclusão'] + df['indicadordeforadoprazo']
        # Calcular o hash para a concatenação das colunas
        hashes_movimentacao = concatenacao_movimentacao.apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        # Adicionar a coluna 'hash' ao DataFrame
        df['sk_movimentacao'] = hashes_movimentacao

        # Renomar colunas
        df = df.rename(columns={
            'competênciamov': 'sk_periodo',
            'município': 'sk_localidade',
            'idade': 'nu_idade',
            'seção': 'is_secao_econ',
            'subclasse': 'is_subclasse_econ',
            'saldomovimentação':'nu_saldo',
            'cbo2002ocupação':'is_cbo_ocupacao',
            'categoria':'is_categoria',
            'graudeinstrução':'is_grau_instrucao',
            'horascontratuais':'nu_hora_contratual',
            'raçacor':'is_etnia',
            'sexo':'is_genero',
            'tipoempregador':'is_tp_empregador',
            'tipoestabelecimento':'is_tp_estabelecimento',
            'tipomovimentação':'is_tp_movimentacao',
            'tipodedeficiência':'is_tp_deficiencia',
            'indtrabintermitente':'is_fl_intermitente',
            'indtrabparcial':'is_fl_trab_parcial',
            'salário':'vl_salario',
            'tamestabjan':'is_faixa_tam_estab',
            'indicadoraprendiz':'is_fl_aprendiz',
            'indicadordeexclusão':'is_fl_exclusao',
            'indicadordeforadoprazo':'is_fl_fora_prazo'
            })

        # Excluir coluna não usadas
        df = df.drop([
            'unidadesaláriocódigo',
            'valorsaláriofixo',
            'região',
            'uf',
            'origemdainformação',
            'competênciadec',
            'competênciaexc'
            ], axis=1)

        # Adiciona o dataframe à lista de dataframes
        dfs.append(df)

# concatena os dataframes em um único dataframe final
df_group = pd.concat(dfs, ignore_index=True)

In [77]:
# exibe o dataframe final
pd.set_option('display.max_columns', None)
df_group.head()

,sk_periodo,sk_localidade,is_secao_econ,is_subclasse_econ,nu_saldo,is_cbo_ocupacao,is_categoria,is_grau_instrucao,nu_idade,nu_hora_contratual,is_etnia,is_genero,is_tp_empregador,is_tp_estabelecimento,is_tp_movimentacao,is_tp_deficiencia,is_fl_intermitente,is_fl_trab_parcial,vl_salario,is_faixa_tam_estab,is_fl_aprendiz,is_fl_exclusao,is_fl_fora_prazo,data_mov,nu_ano,nu_mes,sk_trabalhador,sk_empregador,sk_movimentacao
0,20200101,352930,Saúde Humana e Serviços Sociais,Atividade Médica Ambulatorial Restrita a Consu...,1,"Recepcionista, em Geral",Empregado - Contrato a termo firmado nos termo...,Médio Completo,27,44.0,Branca,Feminino,PJ,CNPJ,Admissão por reemprego,Não Deficiente,Não,Não,1200.0,De 1 a 4,Não,Sim,Não,2020-01-01,2020,1,0e0f011ffb9f94a4f7b8cca9334f33db,51f4696066612a7075946909494147fb,150e0bcf561c91bb54ef8009af14ea88
1,20200101,330630,"Comércio, Reparação de Veículos Automotores e ...",Comércio Varejista de Material Elétrico,-1,Eletricista de Manutencao Eletroeletronica,"Empregado - Geral, inclusive o empregado públi...",Médio Completo,29,42.5,Branca,Masculino,PJ,CNPJ,Desligamento por demissão sem justa causa,Não Deficiente,Não,Não,1836.0,De 20 a 49,Não,Sim,Não,2020-01-01,2020,1,2f71fb6038bd71bef7f4bfaf35ef8ed3,c7a4fab4d064b8aaa98eaff213884aaa,d102ee27edb02420b98246d91ba8db3c
2,20200101,150140,"Comércio, Reparação de Veículos Automotores e ...",Comércio Atacadista de Cosméticos e Produtos d...,-1,Gerente Comercial,"Empregado - Geral, inclusive o empregado públi...",Médio Completo,39,44.0,Parda,Feminino,PJ,CNPJ,Desligamento por demissão sem justa causa,Não Deficiente,Não,Não,1998.0,De 1 a 4,Não,Sim,Não,2020-01-01,2020,1,b55b65f58a1a9976dbef4e548f75049c,1407d505637bbd69228f85ec7f96fb9d,c64f6bda192d6211e03948ed3a7afec1
3,20200101,330455,Saúde Humana e Serviços Sociais,Atividades de Apoio à Gestão de Saúde,-1,Agente Comunitario de Saude,Empregado - Contrato a termo firmado nos termo...,Médio Completo,48,48.0,Preta,Feminino,PJ,CNPJ,Término contrato trabalho prazo determinado,Não Deficiente,Não,Não,1400.0,1000 ou Mais,Não,Sim,Não,2020-01-01,2020,1,44d0fa25808d49819be4d6bc3d0c6134,91ae5f9a0cb8bbb56c9a23494f171720,ca5912a9d921b7e463b34f30a196d601
4,20200101,330455,Saúde Humana e Serviços Sociais,Atividades de Apoio à Gestão de Saúde,-1,Agente Comunitario de Saude,Empregado - Contrato a termo firmado nos termo...,Superior Incompleto,32,48.0,Preta,Feminino,PJ,CNPJ,Término contrato trabalho prazo determinado,Não Deficiente,Não,Não,1400.0,1000 ou Mais,Não,Sim,Não,2020-01-01,2020,1,6d295209cb349a12e7073037ecb32244,91ae5f9a0cb8bbb56c9a23494f171720,ca5912a9d921b7e463b34f30a196d601


In [58]:
pd.set_option('display.max_rows', None)
df_group.dtypes

sk_periodo                       object
sk_localidade                     int64
is_secao_econ                    object
is_subclasse_econ                object
nu_saldo                          int64
is_cbo_ocupacao                  object
is_categoria                     object
is_grau_instrucao                object
nu_idade                          int32
nu_hora_contratual               object
is_etnia                         object
is_genero                        object
is_tp_empregador                 object
is_tp_estabelecimento            object
is_tp_movimentacao               object
is_tp_deficiencia                object
is_fl_intermitente               object
is_fl_trab_parcial               object
vl_salario                      float64
is_faixa_tam_estab               object
is_fl_aprendiz                   object
is_fl_exclusao                   object
is_fl_fora_prazo                 object
data_mov                 datetime64[ns]
nu_ano                            int64


In [59]:
df_group.columns

Index(['sk_periodo', 'sk_localidade', 'is_secao_econ', 'is_subclasse_econ',
       'nu_saldo', 'is_cbo_ocupacao', 'is_categoria', 'is_grau_instrucao',
       'nu_idade', 'nu_hora_contratual', 'is_etnia', 'is_genero',
       'is_tp_empregador', 'is_tp_estabelecimento', 'is_tp_movimentacao',
       'is_tp_deficiencia', 'is_fl_intermitente', 'is_fl_trab_parcial',
       'vl_salario', 'is_faixa_tam_estab', 'is_fl_aprendiz', 'is_fl_exclusao',
       'is_fl_fora_prazo', 'data_mov', 'nu_ano', 'nu_mes', 'sk_trabalhador',
       'sk_empregador', 'sk_movimentacao'],
      dtype='object')

In [60]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948000 entries, 0 to 1947999
Data columns (total 29 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   sk_periodo             object        
 1   sk_localidade          int64         
 2   is_secao_econ          object        
 3   is_subclasse_econ      object        
 4   nu_saldo               int64         
 5   is_cbo_ocupacao        object        
 6   is_categoria           object        
 7   is_grau_instrucao      object        
 8   nu_idade               int32         
 9   nu_hora_contratual     object        
 10  is_etnia               object        
 11  is_genero              object        
 12  is_tp_empregador       object        
 13  is_tp_estabelecimento  object        
 14  is_tp_movimentacao     object        
 15  is_tp_deficiencia      object        
 16  is_fl_intermitente     object        
 17  is_fl_trab_parcial     object        
 18  vl_salario            

In [61]:
df_group.shape

(1948000, 29)

In [62]:
pd.set_option('display.max_rows', None)
df_group.isna().sum()

sk_periodo                  0
sk_localidade               0
is_secao_econ               0
is_subclasse_econ           0
nu_saldo                    0
is_cbo_ocupacao             0
is_categoria                0
is_grau_instrucao           0
nu_idade                    0
nu_hora_contratual       5087
is_etnia                    0
is_genero                   0
is_tp_empregador            0
is_tp_estabelecimento       0
is_tp_movimentacao          0
is_tp_deficiencia           0
is_fl_intermitente          0
is_fl_trab_parcial          0
vl_salario                  0
is_faixa_tam_estab          0
is_fl_aprendiz              0
is_fl_exclusao              0
is_fl_fora_prazo            0
data_mov                    0
nu_ano                      0
nu_mes                      0
sk_trabalhador              0
sk_empregador               0
sk_movimentacao             0
dtype: int64

In [63]:
# Criar df_periodo
df_periodo = df_group[['sk_periodo','data_mov','nu_ano','nu_mes']].drop_duplicates().reset_index(drop=True)
df_periodo.name = 'df_periodo'
df_periodo.head()

,sk_periodo,data_mov,nu_ano,nu_mes
0,20200101,2020-01-01,2020,1
1,20200201,2020-02-01,2020,2
2,20200301,2020-03-01,2020,3
3,20200401,2020-04-01,2020,4
4,20200501,2020-05-01,2020,5


In [64]:
df_periodo.dtypes

sk_periodo            object
data_mov      datetime64[ns]
nu_ano                 int64
nu_mes                 int64
dtype: object

In [65]:
# Define o caminho para a pasta "dados"
folder_path = "dados"


df_localidade = pd.read_csv(os.path.join(folder_path, 'MUNICIPIO.csv'), delimiter=';', encoding='ISO-8859-1')
# Renomear as colunas
df_localidade = df_localidade.rename(columns={'nm_regiao': 'is_regiao','nm_uf': 'is_uf','cd_mun_ibge_6': 'sk_localidade','nm_municipio': 'is_municipio'})
# Excluir colunas não usadas
df_localidade = df_localidade.drop(['cd_regiao','cd_uf','cd_mun_ibge_7'], axis=1)
# Corrigir a sequência
df_localidade = df_localidade.reindex(columns=['sk_localidade', 'is_regiao', 'is_uf','is_municipio'])

In [66]:
df_localidade.head()

,sk_localidade,is_regiao,is_uf,is_municipio
0,110001,Norte,Rondônia,Alta Floresta D'Oeste
1,110037,Norte,Rondônia,Alto Alegre dos Parecis
2,110040,Norte,Rondônia,Alto Paraíso
3,110034,Norte,Rondônia,Alvorada D'Oeste
4,110002,Norte,Rondônia,Ariquemes


In [67]:
df_movimentacao =  df_group[[
    'sk_movimentacao','is_tp_movimentacao','is_categoria','is_cbo_ocupacao','is_fl_intermitente',
    'is_fl_trab_parcial','is_fl_aprendiz','is_fl_exclusao','is_fl_fora_prazo'
    ]].drop_duplicates().reset_index(drop=True)

In [68]:
df_movimentacao.head()

,sk_movimentacao,is_tp_movimentacao,is_categoria,is_cbo_ocupacao,is_fl_intermitente,is_fl_trab_parcial,is_fl_aprendiz,is_fl_exclusao,is_fl_fora_prazo
0,150e0bcf561c91bb54ef8009af14ea88,Admissão por reemprego,Empregado - Contrato a termo firmado nos termo...,"Recepcionista, em Geral",Não,Não,Não,Sim,Não
1,d102ee27edb02420b98246d91ba8db3c,Desligamento por demissão sem justa causa,"Empregado - Geral, inclusive o empregado públi...",Eletricista de Manutencao Eletroeletronica,Não,Não,Não,Sim,Não
2,c64f6bda192d6211e03948ed3a7afec1,Desligamento por demissão sem justa causa,"Empregado - Geral, inclusive o empregado públi...",Gerente Comercial,Não,Não,Não,Sim,Não
3,ca5912a9d921b7e463b34f30a196d601,Término contrato trabalho prazo determinado,Empregado - Contrato a termo firmado nos termo...,Agente Comunitario de Saude,Não,Não,Não,Sim,Não
4,78cd46c12ef15764b729bbce5eecae10,Admissão por reemprego,"Empregado - Geral, inclusive o empregado públi...",Administrador de Edificios,Não,Não,Não,Sim,Sim


In [69]:
df_trabalhador =  df_group[[
    'sk_trabalhador','is_grau_instrucao','is_genero','nu_idade','is_etnia','is_tp_deficiencia'
    ]].drop_duplicates().reset_index(drop=True)

In [70]:
df_trabalhador.head()

,sk_trabalhador,is_grau_instrucao,is_genero,nu_idade,is_etnia,is_tp_deficiencia
0,0e0f011ffb9f94a4f7b8cca9334f33db,Médio Completo,Feminino,27,Branca,Não Deficiente
1,2f71fb6038bd71bef7f4bfaf35ef8ed3,Médio Completo,Masculino,29,Branca,Não Deficiente
2,b55b65f58a1a9976dbef4e548f75049c,Médio Completo,Feminino,39,Parda,Não Deficiente
3,44d0fa25808d49819be4d6bc3d0c6134,Médio Completo,Feminino,48,Preta,Não Deficiente
4,6d295209cb349a12e7073037ecb32244,Superior Incompleto,Feminino,32,Preta,Não Deficiente


In [71]:
df_empregador =  df_group[[
    'sk_empregador','is_tp_empregador','is_tp_estabelecimento','is_secao_econ','is_subclasse_econ','is_faixa_tam_estab'
    ]].drop_duplicates().reset_index(drop=True)

In [72]:
df_empregador.head()

,sk_empregador,is_tp_empregador,is_tp_estabelecimento,is_secao_econ,is_subclasse_econ,is_faixa_tam_estab
0,51f4696066612a7075946909494147fb,PJ,CNPJ,Saúde Humana e Serviços Sociais,Atividade Médica Ambulatorial Restrita a Consu...,De 1 a 4
1,c7a4fab4d064b8aaa98eaff213884aaa,PJ,CNPJ,"Comércio, Reparação de Veículos Automotores e ...",Comércio Varejista de Material Elétrico,De 20 a 49
2,1407d505637bbd69228f85ec7f96fb9d,PJ,CNPJ,"Comércio, Reparação de Veículos Automotores e ...",Comércio Atacadista de Cosméticos e Produtos d...,De 1 a 4
3,91ae5f9a0cb8bbb56c9a23494f171720,PJ,CNPJ,Saúde Humana e Serviços Sociais,Atividades de Apoio à Gestão de Saúde,1000 ou Mais
4,fe0ace4b938b0743442591be2f43199b,PJ,CNPJ,Atividades Administrativas e Serviços Compleme...,Serviços Combinados de Escritório e Apoio Admi...,De 1 a 4


In [73]:
df_fato_caged = df_group[[
    'sk_empregador',
    'sk_periodo',
    'sk_trabalhador',
    'sk_movimentacao',
    'sk_localidade',
    'vl_salario',
    'nu_hora_contratual',
    'nu_saldo'
    ]]

In [74]:
df_fato_caged.head()

,sk_empregador,sk_periodo,sk_trabalhador,sk_movimentacao,sk_localidade,vl_salario,nu_hora_contratual,nu_saldo
0,51f4696066612a7075946909494147fb,20200101,0e0f011ffb9f94a4f7b8cca9334f33db,150e0bcf561c91bb54ef8009af14ea88,352930,1200.0,"44,00",1
1,c7a4fab4d064b8aaa98eaff213884aaa,20200101,2f71fb6038bd71bef7f4bfaf35ef8ed3,d102ee27edb02420b98246d91ba8db3c,330630,1836.0,"42,50",-1
2,1407d505637bbd69228f85ec7f96fb9d,20200101,b55b65f58a1a9976dbef4e548f75049c,c64f6bda192d6211e03948ed3a7afec1,150140,1998.0,"44,00",-1
3,91ae5f9a0cb8bbb56c9a23494f171720,20200101,44d0fa25808d49819be4d6bc3d0c6134,ca5912a9d921b7e463b34f30a196d601,330455,1400.0,"48,00",-1
4,91ae5f9a0cb8bbb56c9a23494f171720,20200101,6d295209cb349a12e7073037ecb32244,ca5912a9d921b7e463b34f30a196d601,330455,1400.0,"48,00",-1
